https://calculatorhistory.net

In [1]:
sealed interface Entry {
    enum class BinOp: Entry { ADD, MULT }
    enum class Delim: Entry { LPAREN, RPAREN }
    data class Value(val number: Int): Entry
}

In [2]:
Entry.BinOp.ADD

ADD

In [3]:
Entry.Delim.LPAREN

LPAREN

In [4]:
Entry.Value(43)

Value(number=43)

In [5]:
interface ICalculator {
    fun appendEntry(e: Entry): Boolean // adds an entriy (either BinOp, Delim, or Vaule) at the end
        // appends legal entry input and return true (returns false for illegal input without adding it)
    fun removeEntry(): Boolean  // removes the last added entry; ◄(x or DEL) button
        // removes the last input entry and returns true (returns false if not possible)
    fun calculate(): Entry.Value?    // push the = button
        // calculate current entry and returns the result value (return null when it fails) 
        // add to history
        // clear current entries and nullify last calculated value
    fun currValue(): Entry.Value? // currently calculated value from the current entries (defaults to null)
    fun lastValue(): Entry.Value  // last calculated value (defaults to 0)
    fun entries(): List<Entry> // get current entries
    fun history(): List< Pair<List<Entry>, Entry.Value> > // history of calculations
    fun clearEntries() // clear the current entries; AC button
    fun clearHistory() // clear history
}

In [6]:
// ... class Calculator: ICalculator ...

----
참고자료: https://madplay.github.io/post/call-javascript-function-from-java-using-graalvm

In [7]:
@file:DependsOn("org.graalvm.js:js:22.0.0.2")

In [8]:
import org.graalvm.polyglot.*

val context = Context.create("js"); // 그냥 항상 성공한다고 가정하면 이렇게 써도 ...

val value = context.eval("js", "3 + 4 * 10");
println( value );
println( value.toString() == "43" )
println( value::class )

43
true
class org.graalvm.polyglot.Value


In [9]:
import org.graalvm.polyglot.*

val context = Context.create("js"); // 그냥 항상 성공한다고 가정하면 이렇게 써도 ...

try {
    val value = context.eval("js", "3 + 4 * ");
    println( value );
    println( value.toString() == "43" )
    println( value::class )
} catch (e : Exception) {
    println(e)
    println("이걸 적당히 잘 처리하면 유용하게 활용 가능")
}

org.graalvm.polyglot.PolyglotException: SyntaxError: Unnamed:1:8 Expected an operand but found eof
3 + 4 * 
        ^

이걸 적당히 잘 처리하면 유용하게 활용 가능
